In [ ]:
import pandas as pd
import random
from itertools import combinations
import csv
import math
import time
random.seed(time.time())


class MedicalDataAugmentation:
    def __init__(self):
        # Các từ nối tự nhiên giữa các triệu chứng
        self.connection_words = ["và", "cùng với", "kèm theo", "đồng thời", "ngoài ra còn"]

        # Các mẫu câu tự nhiên
        self.sentence_patterns = [
            "tôi bị {symptoms}",
            "tôi có triệu chứng {symptoms}",
            "tôi đang gặp phải {symptoms}",
            "tôi cảm thấy {symptoms}",
            "tôi xuất hiện {symptoms}",
            "tôi có dấu hiệu {symptoms}",
            "tôi gặp phải {symptoms}",
            "tôi đang có {symptoms}",
            "tôi phát hiện {symptoms}"
        ]

    def load_data(self, file_path):
        """Đọc dữ liệu từ file Excel"""
        try:
            df = pd.read_excel(file_path)
            print(f"✅ Đã đọc {len(df)} bệnh từ file: {file_path}")
            return df
        except Exception as e:
            print(f"❌ Lỗi khi đọc file: {e}")
            return None

    def parse_symptoms(self, symptoms_text):
        """Tách các triệu chứng từ chuỗi text"""
        return [s.strip() for s in symptoms_text.split(',') if s.strip()]

    def shuffle_symptoms(self, symptom_tuple):
        symptoms = list(symptom_tuple)
        random.shuffle(symptoms)
        return symptoms


    def create_symptom_combinations(self, symptoms, ratio=2/3):
        """Tạo các tổ hợp triệu chứng"""
        if not symptoms:
            return []

        total_symptoms = len(symptoms)
        num_to_select = max(1, math.ceil(total_symptoms * ratio))

        # Nếu triệu chứng ít hơn số cần chọn, thì trả về nguyên triệu chứng
        if total_symptoms <= num_to_select:
            return [tuple(symptoms)]

        # Tạo tất cả tổ hợp độ dài num_to_select
        return list(combinations(symptoms, num_to_select))

    def create_natural_sentence(self, symptoms_list):
        """Tạo câu tự nhiên từ danh sách triệu chứng"""
        if not symptoms_list:
            return ""

        pattern = random.choice(self.sentence_patterns)

        if len(symptoms_list) == 1:
            symptoms_text = symptoms_list[0]
        else:
            connector = random.choice(self.connection_words)
            symptoms_text = f"{symptoms_list[0]} {connector} {', '.join(symptoms_list[1:])}"

        return pattern.format(symptoms=symptoms_text)

    def shuffle_symptoms(self, symptom_tuple):
        symptoms = list(symptom_tuple)
        random.shuffle(symptoms)
        return symptoms


    def augment_disease_data(self, disease_name, symptoms_text, target_samples=20):
        """Tạo dữ liệu tăng cường cho một bệnh"""
        symptoms = self.parse_symptoms(symptoms_text)
        print(f"🦠 Bệnh: {disease_name} - {len(symptoms)} triệu chứng")

        combinations_list = self.create_symptom_combinations(self.shuffle_symptoms(symptoms), ratio=1/4)

        samples = []
        used_combinations = set()

        for _ in range(target_samples):
            if combinations_list:
                available = [c for c in combinations_list if c not in used_combinations]
                if not available:
                    available = combinations_list
                    used_combinations.clear()

                selected = random.choice(available)
                used_combinations.add(selected)
                sentence = self.create_natural_sentence(list(selected))
                samples.append({
                    'disease': disease_name,
                    'symptoms': sentence,
                    'original_symptoms': ', '.join(selected)
                })
            else:
                # fallback: dùng hết triệu chứng gốc
                sentence = self.create_natural_sentence(symptoms)
                samples.append({
                    'disease': disease_name,
                    'symptoms': sentence,
                    'original_symptoms': ', '.join(symptoms)
                })

        return samples


    def process_all_diseases(self, df, output_file='./data_processed/' + 'augmented_medical_data.csv'):
        """Tăng cường dữ liệu cho tất cả các bệnh và lưu vào CSV"""
        all_samples = []

        for index, row in df.iterrows():
            disease_name = row.iloc[0]
            symptoms_text = row.iloc[1]

            print(f"\n🔄 Đang xử lý: {disease_name}")
            samples = self.augment_disease_data(disease_name, symptoms_text, target_samples=20)
            all_samples.extend(samples)
            print(f"✅ Đã tạo {len(samples)} câu cho {disease_name}")

        # Lưu file CSV
        with open(output_file, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['disease', 'symptoms', 'original_symptoms'])
            writer.writeheader()
            writer.writerows(all_samples)

        print(f"\n📁 Đã lưu {len(all_samples)} câu vào file: {output_file}")
        return all_samples


In [ ]:
processor = MedicalDataAugmentation()

# Đọc dữ liệu từ file Excel
df = processor.load_data('./data/Trieu_chung.xlsx')

if df is not None:
    # Xử lý và tạo dữ liệu tăng cường
    samples = processor.process_all_diseases(df)

    # Hiển thị thống kê
    print("\n=== THỐNG KÊ DỮ LIỆU ===")
    disease_counts = {}
    for sample in samples:
        disease = sample['disease']
        disease_counts[disease] = disease_counts.get(disease, 0) + 1

    for disease, count in disease_counts.items():
        print(f"{disease}: {count} samples")

    print(f"\nTổng cộng: {len(samples)} samples")

    # Hiển thị một số mẫu
    print("\n=== MỘT SỐ MẪU DỮ LIỆU ===")
    for i, sample in enumerate(samples[:5]):
        print(f"Sample {i+1}:")
        print(f"  Bệnh: {sample['disease']}")
        print(f"  Triệu chứng: {sample['symptoms']}")
        print(f"  Triệu chứng gốc: {sample['original_symptoms']}")
        print()

✅ Đã đọc 29 bệnh từ file: /content/drive/MyDrive/Colab Notebooks/NLP/health_api/data/Trieu_chung.xlsx

🔄 Đang xử lý: táo bón
🦠 Bệnh: táo bón - 9 triệu chứng
✅ Đã tạo 20 câu cho táo bón

🔄 Đang xử lý: viêm mũi họng
🦠 Bệnh: viêm mũi họng - 13 triệu chứng
✅ Đã tạo 20 câu cho viêm mũi họng

🔄 Đang xử lý: khô mắt
🦠 Bệnh: khô mắt - 6 triệu chứng
✅ Đã tạo 20 câu cho khô mắt

🔄 Đang xử lý: hạ canxi máu
🦠 Bệnh: hạ canxi máu - 8 triệu chứng
✅ Đã tạo 20 câu cho hạ canxi máu

🔄 Đang xử lý: sốt phát ban
🦠 Bệnh: sốt phát ban - 11 triệu chứng
✅ Đã tạo 20 câu cho sốt phát ban

🔄 Đang xử lý: viêm xương
🦠 Bệnh: viêm xương - 17 triệu chứng
✅ Đã tạo 20 câu cho viêm xương

🔄 Đang xử lý: viêm khớp
🦠 Bệnh: viêm khớp - 11 triệu chứng
✅ Đã tạo 20 câu cho viêm khớp

🔄 Đang xử lý: đau dạ dày
🦠 Bệnh: đau dạ dày - 12 triệu chứng
✅ Đã tạo 20 câu cho đau dạ dày

🔄 Đang xử lý: viêm xoang
🦠 Bệnh: viêm xoang - 13 triệu chứng
✅ Đã tạo 20 câu cho viêm xoang

🔄 Đang xử lý: viêm mũi dị ứng
🦠 Bệnh: viêm mũi dị ứng - 13 triệ